In [1]:
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
os.chdir('/content/drive/MyDrive/sk_llm_text2sql/')

In [4]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 77.9 MB/s eta 0:00:00
Processing /colabtools/dist/google-colab-1.0.0.tar.gz (from -r requirements.txt (line 127))
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/colabtools/dist/google-colab-1.0.0.tar.gz'



In [5]:
import pandas as pd
import yaml

import clickhouse_connect
from langchain import LLMChain, PromptTemplate
from langchain.llms import OpenAI
from connector import ClickhouseConnector
from text2sql import PromptSchemaTransformer, Text2Sql

In [8]:
if __name__ == 'main':

  config_path = 'config/'
  with open(f"{config_path}config.yaml", "r") as stream:
    try:
        config = yaml.safe_load(stream)

        connections_config = config['connections']
        clickhouse_config = connections_config['clickhouse']
        clickhouse_host = clickhouse_config['host']
        clickhouse_username = clickhouse_config['username']
        clickhouse_password = clickhouse_config['password']
        clickhouse_port = clickhouse_config['port']

        models_config = config['models']
        openai_config = models_config['openai']
        openai_api_key = openai_config['api_key']
        openai_params = openai_config['params']
    except yaml.YAMLError as exc:
        print(exc)

    os.environ["OPENAI_API_KEY"] = openai_api_key

    conn = ClickhouseConnector(clickhouse_host, clickhouse_username, clickhouse_password, clickhouse_port)
    client = conn.create_connection()

    pst = PromptSchemaTransformer(client)
    db_schema = pst.transform_db_schema_for_prompt()

    model = OpenAI(**openai_params)
    text2sql = Text2Sql(model)

    question = 'How many LinkedIn clicks did we have in 2022?'

    sql = text2sql.get_sql_by_question_and_schema(question, db_schema)

    df = client.query_df(f"""{sql}""")
    print(df)